<a href="https://colab.research.google.com/github/shariqfz/Fake-News-Detection-Text-Classification/blob/main/fake_news_detection_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import string

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.linear_model import LogisticRegression



In [34]:
!mkdir -p data
!git clone 'https://github.com/shariqfz/Fake-News-Detection-Text-Classification.git'
!tar -zxvf '/content/Fake-News-Detection-Text-Classification/data/FakeNewsData.tar.gz'
!mv ./FakeNewsData/Fake.csv ./FakeNewsData/True.csv ./data
!rm -r /content/FakeNewsData /content/sample_data

fatal: destination path 'Fake-News-Detection-Text-Classification' already exists and is not an empty directory.
FakeNewsData/
FakeNewsData/True.csv
FakeNewsData/Fake.csv
FakeNewsData/Fake.csv:Zone.Identifier
FakeNewsData/True.csv:Zone.Identifier
rm: cannot remove '/content/sample_data': No such file or directory


In [35]:
fake_news_path = "/content/data/True.csv"
real_news_path = "/content/data/Fake.csv"

In [36]:
fake_news = pd.read_csv(fake_news_path)
real_news = pd.read_csv(real_news_path)

In [37]:
fake_news.head(3)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"


In [38]:

real_news.head(3)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"


In [39]:
real = real_news.copy()
fake = fake_news.copy()
real['impression'] = 'Real'
fake['impression'] = 'Fake'

news = pd.concat([real, fake], axis=0, ignore_index=True)

print(f"Samples available: {news.shape[0]}\n#features of dataset: {news.shape[1]}")

Samples available: 44898
#features of dataset: 5


In [40]:
news['impression'] = news['impression'].map({'Fake': 0, 'Real': 1})

In [41]:
news['fulltext'] = news['title'] + ' ' + news['text']
news.drop(['title', 'text'], axis=1, inplace=True)


In [42]:
data = news[['fulltext', 'impression']]


In [43]:
print(data.isnull().sum())

fulltext      0
impression    0
dtype: int64


In [44]:
data_sample = data.sample(n=10000, random_state=1)
X_data = data_sample['fulltext'].astype(str)
y_data = data_sample['impression']

In [45]:
def preprocess_data(X_data_full):
    # Preprocessing steps here...

    # Create features using CountVectorizer and TfidfTransformer
    cv = CountVectorizer(max_features=1000)
    X_data_full_vector = cv.fit_transform(X_data_full).toarray()

    tfidf = TfidfTransformer()
    X_data_full_tfidf = tfidf.fit_transform(X_data_full_vector).toarray()

    return X_data_full_tfidf


In [46]:
data_X = preprocess_data(X_data)
seed=6000

In [47]:

X_train, X_test, y_train, y_test = train_test_split(data_X, y_data, test_size=0.20, random_state=seed)

In [48]:
logistic_reg = LogisticRegression(random_state=seed)
logistic_reg.fit(X_train, y_train)

LogisticRegression(random_state=6000)

In [49]:
predictions = logistic_reg.predict(X_test)

In [50]:
print(classification_report(y_test, predictions))

logistic_reg_f1 = round(f1_score(y_test, predictions, average='weighted'), 3)
logistic_reg_accuracy = round((accuracy_score(y_test, predictions) * 100), 2)

print("Accuracy : ", logistic_reg_accuracy, " %")
print("f1_score : ", logistic_reg_f1)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       966
           1       0.98      0.98      0.98      1034

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000

Accuracy :  97.7  %
f1_score :  0.977
